In [186]:
import numpy as np
import pandas as pd
import pybaseball
import seaborn as sns
import matplotlib.pyplot as plot
from deepdiff import DeepDiff
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register
from pybaseball import pitching_stats
from pybaseball import batting_stats
from time import sleep

cache.enable()
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 10)

# These are the people

In [187]:
biofile = pd.read_csv('/Users/bean/Dropbox/code/_projects/baseball/_data/retrosheet/biofile.txt')

In [188]:
biofile = biofile.convert_dtypes()

In [189]:
biofile[['FEET', 'INCHES']] = biofile['HEIGHT'].str.split('-', 1, expand=True)

In [190]:
biofile['FEET'] = biofile['FEET'].astype(float)
biofile['INCHES'] = biofile['FEET'].astype(float)

In [191]:
biofile.INCHES.dtype

dtype('float64')

In [192]:
biofile['BMI'] = ((biofile['WEIGHT'] * KG_TO_LB) / (((biofile['FEET'] * 12) + biofile['INCHES']) * INCH_TO_METER) ** 2)

In [193]:
biofile = biofile.convert_dtypes()
biofile_small = biofile[['PLAYERID', 'FIRST', 'LAST', 'INCHES', 'WEIGHT', 'BMI']]

In [194]:
biofile_small.describe().to_csv('biofile_small.csv')

In [195]:
KG_TO_LB = 0.4536
INCH_TO_METER = .0254

In [196]:
biofile_small.BMI.describe()

count        19840
mean           NaN
std           <NA>
min            NaN
25%       21.49422
50%      23.689866
75%      27.457133
max            NaN
Name: BMI, dtype: object

In [197]:
topweight = biofile.loc[biofile['WEIGHT'] == biofile['WEIGHT'].max()]
botweight = biofile.loc[biofile['WEIGHT'] == biofile['WEIGHT'].min()]

In [198]:
botweight

,PLAYERID,LAST,FIRST,NICKNAME,BIRTHDATE,BIRTH CITY,BIRTH STATE,BIRTH COUNTRY,PLAY DEBUT,PLAY LASTGAME,MGR DEBUT,MGR LASTGAME,COACH DEBUT,COACH LASTGAME,UMP DEBUT,UMP LASTGAME,DEATHDATE,DEATH CITY,DEATH STATE,DEATH COUNTRY,BATS,THROWS,HEIGHT,WEIGHT,CEMETERY,CEME CITY,CEME STATE,CEME COUNTRY,CEME NOTE,BIRTH NAME,NAME CHG,BAT CHG,HOF,FEET,INCHES,BMI
6155,gaede101,Gaedel,Edward Carl,Eddie,06/08/1925,Chicago,Illinois,USA,08/19/1951,08/19/1951,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,06/18/1961,Chicago,Illinois,USA,R,L,3-07,65,St. Mary Cemetery,Evergreen Park,Illinois,USA,<NA>,Edward Carl Gaedele,<NA>,<NA>,NOT,3,3,30.045684


In [199]:
topweight

,PLAYERID,LAST,FIRST,NICKNAME,BIRTHDATE,BIRTH CITY,BIRTH STATE,BIRTH COUNTRY,PLAY DEBUT,PLAY LASTGAME,MGR DEBUT,MGR LASTGAME,COACH DEBUT,COACH LASTGAME,UMP DEBUT,UMP LASTGAME,DEATHDATE,DEATH CITY,DEATH STATE,DEATH COUNTRY,BATS,THROWS,HEIGHT,WEIGHT,CEMETERY,CEME CITY,CEME STATE,CEME COUNTRY,CEME NOTE,BIRTH NAME,NAME CHG,BAT CHG,HOF,FEET,INCHES,BMI
21177,mcshj901,McSherry,John Patrick,John,09/11/1944,Bronx,New York,USA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,06/01/1971,10/01/1995,04/01/1996,Cincinnati,Ohio,USA,<NA>,<NA>,6-03,351,Cemetery of the Gate of Heaven,Hawthorne,New York,USA,"Section 44, Plot 480, Grave 3; N41.05.184 W73....",<NA>,<NA>,<NA>,NOT,6,6,40.561673


In [200]:
botweight

,PLAYERID,LAST,FIRST,NICKNAME,BIRTHDATE,BIRTH CITY,BIRTH STATE,BIRTH COUNTRY,PLAY DEBUT,PLAY LASTGAME,MGR DEBUT,MGR LASTGAME,COACH DEBUT,COACH LASTGAME,UMP DEBUT,UMP LASTGAME,DEATHDATE,DEATH CITY,DEATH STATE,DEATH COUNTRY,BATS,THROWS,HEIGHT,WEIGHT,CEMETERY,CEME CITY,CEME STATE,CEME COUNTRY,CEME NOTE,BIRTH NAME,NAME CHG,BAT CHG,HOF,FEET,INCHES,BMI
6155,gaede101,Gaedel,Edward Carl,Eddie,06/08/1925,Chicago,Illinois,USA,08/19/1951,08/19/1951,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,06/18/1961,Chicago,Illinois,USA,R,L,3-07,65,St. Mary Cemetery,Evergreen Park,Illinois,USA,<NA>,Edward Carl Gaedele,<NA>,<NA>,NOT,3,3,30.045684


In [201]:
biofile.loc[biofile['PLAYERID'] == 'troum001']

,PLAYERID,LAST,FIRST,NICKNAME,BIRTHDATE,BIRTH CITY,BIRTH STATE,BIRTH COUNTRY,PLAY DEBUT,PLAY LASTGAME,MGR DEBUT,MGR LASTGAME,COACH DEBUT,COACH LASTGAME,UMP DEBUT,UMP LASTGAME,DEATHDATE,DEATH CITY,DEATH STATE,DEATH COUNTRY,BATS,THROWS,HEIGHT,WEIGHT,CEMETERY,CEME CITY,CEME STATE,CEME COUNTRY,CEME NOTE,BIRTH NAME,NAME CHG,BAT CHG,HOF,FEET,INCHES,BMI
18405,troum001,Trout,Michael Nelson,Mike,08/07/1991,Vineland,New Jersey,USA,07/08/2011,05/17/2021,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,R,R,6-01,200,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NOT,6,6,23.112065


In [202]:
biofile.groupby('HEIGHT').describe()

WEIGHT                                                              \
        count        mean        std    min     25%    50%     75%    max   
HEIGHT                                                                      
3-07        1        65.0       <NA>     65      65     65      65     65   
36678       1       200.0       <NA>    200     200    200     200    200   
44325       1       140.0       <NA>    140     140    140     140    140   
44327       1       194.0       <NA>    194     194    194     194    194   
5-03     10.0       144.5  14.222439  125.0  135.75  142.5   155.0  170.0   
...       ...         ...        ...    ...     ...    ...     ...    ...   
6-07.5      1       235.0       <NA>    235     235    235     235    235   
6-08     37.0  226.810811  18.133514  185.0   220.0  225.0   240.0  265.0   
6-09     12.0  237.833333  27.185669  180.0   225.0  242.0   255.0  280.0   
6-10      6.0  244.166667  17.724747  225.0  231.25  240.0  256.25  270.0   
6-11        1       230.0       <NA>    230     230    230     230    230   

       NAME CHG                                           BAT CHG              \
          count  mean   std   min   25%   50%   75%   max   count  mean   std   
HEIGHT                                                                          
3-07          0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
36678         0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
44325         0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
44327         0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
5-03          0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
...         ...   ...   ...   ...   ...   ...   ...   ...     ...   ...   ...   
6-07.5        0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
6-08          0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
6-09          0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
6-10          0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   
6-11          0  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>       0  <NA>  <NA>   

                                      FEET                               \
         min   25%   50%   75%   max count     mean   std    min    25%   
HEIGHT                                                                    
3-07    <NA>  <NA>  <NA>  <NA>  <NA>     1      3.0  <NA>      3      3   
36678   <NA>  <NA>  <NA>  <NA>  <NA>     1  36678.0  <NA>  36678  36678   
44325   <NA>  <NA>  <NA>  <NA>  <NA>     1  44325.0  <NA>  44325  44325   
44327   <NA>  <NA>  <NA>  <NA>  <NA>     1  44327.0  <NA>  44327  44327   
5-03    <NA>  <NA>  <NA>  <NA>  <NA>  10.0      5.0   0.0    5.0    5.0   
...      ...   ...   ...   ...   ...   ...      ...   ...    ...    ...   
6-07.5  <NA>  <NA>  <NA>  <NA>  <NA>     1      6.0  <NA>      6      6   
6-08    <NA>  <NA>  <NA>  <NA>  <NA>  37.0      6.0   0.0    6.0    6.0   
6-09    <NA>  <NA>  <NA>  <NA>  <NA>  12.0      6.0   0.0    6.0    6.0   
6-10    <NA>  <NA>  <NA>  <NA>  <NA>   6.0      6.0   0.0    6.0    6.0   
6-11    <NA>  <NA>  <NA>  <NA>  <NA>     1      6.0  <NA>      6      6   

                            INCHES                                             \
          50%    75%    max  count     mean   std    min    25%    50%    75%   
HEIGHT                                                                          
3-07        3      3      3      1      3.0  <NA>      3      3      3      3   
36678   36678  36678  36678      1  36678.0  <NA>  36678  36678  36678  36678   
44325   44325  44325  44325      1  44325.0  <NA>  44325  44325  44325  44325   
44327   44327  44327  44327      1  44327.0  <NA>  44327  44327  44327  44327   
5-03      5.0    5.0    5.0   10.0      5.0   0.0    5.0    5.0    5.0    5.0   
...       ...    ...    ...    ...      ...   ...    ...    ...    ...    ...   
6-07.5      6

In [203]:
people = pd.DataFrame(people())

sleep(1)  
# sleep calls to prevent the pybaseball scraper 
# from throwing errors when importing tons of stuff

people = people.convert_dtypes() # cleanup; thank you NumPy

people = people[people.weight.notna()]

print(people.shape)
print("---")
people.sample(3)

(19729, 24)
---


,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
3432,cocopa01,1977,9,8,D.R.,Distrito Nacional,Santo Domingo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Pasqual,Coco,Pasqual,185,73,R,R,2000-07-17,2002-04-14,cocop001,cocopa01
9084,jewettr99,1964,3,3,USA,TX,Dallas,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Trent,Jewett,Phillip Trent,195,74,R,R,<NA>,<NA>,jewet801,<NA>
8337,hittbr01,1897,3,14,USA,TX,Comanche,1973,11,10,USA,OR,Portland,Bruce,Hitt,Bruce Smith,190,73,R,R,1917-09-23,1917-09-30,hittb101,hittbr01


We need to add more ID info about them so that we don't drown in merge errors.

In [204]:
rosetta = pd.DataFrame(chadwick_register()) 

sleep(1)

rosetta = rosetta.convert_dtypes()

rosetta = rosetta.dropna(how='any')

print(rosetta.shape)
print('---')
rosetta.sample(3)

(20506, 8)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
5729,Donnelly,Jim,113448,donnj101,donneji01,1003385,1884,1898
23524,Williams,Rick,124334,willr101,williri03,1014035,1978,1979
15861,Ni,Fu-Te,547820,ni--f001,nifu01,4199,2009,2010


## Merge people into rosetta

In [205]:
everyone = pd.merge(rosetta, people, left_on='key_bbref', right_on='playerID', how='left')

print(everyone.shape)
print('---')
everyone.sample(3)

(20506, 32)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
10353,Lambert,Gene,117433,lambg101,lambege01,1007274,1941,1942,lambege01,1921,4,26,USA,MS,Crenshaw,2000,2,10,USA,TN,Germantown,Gene,Lambert,Eugene Marion,175,71,R,R,1941-09-14,1942-04-26,lambg101,lambege01
4896,Dobb,John,113403,dobbj102,dobbjo01,1003339,1924,1924,dobbjo01,1901,11,15,USA,MI,Muskegon,1991,7,31,USA,MI,Muskegon,John,Dobb,John Kenneth,180,74,R,L,1924-08-13,1924-08-30,dobbj102,dobbjo01
18602,Toms,Tommy,123393,tomst101,tomsto01,1013115,1975,1977,tomsto01,1951,10,15,USA,VA,Charlottesville,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Tommy,Toms,Thomas Howard,195,76,R,R,1975-05-04,1977-07-02,tomst101,tomsto01


In [206]:
# Constants
KG_TO_LB = 0.453592
M_TO_IN = 0.0254

# BMI Calculations
everyone['KG'] = everyone['weight'] * KG_TO_LB
everyone['meters'] = everyone['height'] * M_TO_IN
everyone['BMI'] = everyone['KG'] / everyone['meters'] ** 2
everyone['ratio'] = everyone['meters'] * everyone['BMI']

## Init Fielding Data

In [207]:
# fielding stats by year 
fielding = pd.DataFrame(fielding()) # this is fine

fielding = fielding.convert_dtypes()

In [208]:
fielding = fielding.rename(columns={"playerID": "key_bbref"})

In [209]:
everyone = everyone[
    [
        "name_last",
        "name_first",
        "key_mlbam",
        "key_retro",
        "key_bbref",
        "key_fangraphs",
        "birthYear",
        "deathYear",
        "weight",
        "height",
        "BMI",
        "bats",
        "throws",
    ]
]

In [210]:
biocomp = everyone[
    [
        "weight",
        "height",
        "BMI",
    ]
]

In [211]:
biocomp.describe().to_csv('biocomp.csv')

In [212]:
everyone.describe().to_csv('everyone_describe.csv')

In [213]:
df = everyone
df = fielding.merge(df, on='key_bbref', how='outer', validate='many_to_one')

## Batting data

In [214]:
# batting stats by year 
batting = pd.DataFrame(batting()) # this is fine

batting = batting.convert_dtypes()
batting.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)


,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
12629,aitchra01,1911,1,BRO,NL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12700,camniha01,1911,1,SLN,NL,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12738,cottren01,1911,1,PIT,NL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12830,griffha01,1911,1,CHN,NL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13045,pucketr01,1911,1,PHI,NL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110490,zimmejo02,2021,1,MIL,NL,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
110491,zimmeky01,2021,1,KCA,AL,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
110492,zimmery01,2021,1,WAS,NL,110,255,27,62,16,0,14,46,0,0,16,77,0,0,0,2,9
110493,zuberty01,2021,1,KCA,AL,31,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [215]:
batting.sample(10)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
85789,sizemgr01,2004,1,CLE,AL,43,138,15,34,6,2,4,24,2,0,14,34,0,5,0,2,0
98649,chacijh01,2014,1,COL,NL,12,15,2,5,1,0,0,0,0,0,1,4,0,0,1,0,0
103974,rodrise01,2017,1,ATL,NL,15,37,6,6,1,0,2,3,1,0,8,19,1,1,1,0,1
67762,gerenbo01,1990,1,NYA,AL,110,277,21,59,7,0,8,31,0,0,13,73,1,5,6,2,7
96988,wilsobr01,2012,1,SFN,NL,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
91248,seaybo01,2008,1,DET,AL,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80137,myettaa01,2000,1,CHA,AL,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23014,cochrmi01,1929,1,PHA,AL,135,514,113,170,37,8,7,95,7,6,69,8,<NA>,2,21,<NA>,<NA>
64128,stewada01,1986,2,OAK,AL,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22213,orsater01,1927,1,SLN,NL,27,92,15,29,7,3,0,12,2,<NA>,11,12,<NA>,0,2,<NA>,<NA>


In [216]:
batting.describe().to_csv('mlb_batting.csv')

In [217]:
batting = batting.rename(columns={"playerID": "key_bbref"})

### Merge Batting Data

In [218]:
a = np.intersect1d(df.columns, batting.columns)
print(list(a))

['CS', 'G', 'SB', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID']


In [219]:
bats = df.merge(batting, on=['CS', 'G', 'SB', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator=True)


In [220]:
bats

,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,BMI,bats,throws,AB,R,H,2B,3B,HR,RBI,BB,SO,IBB,HBP,SH,SF,GIDP,_merge
0,abercda01,1871,1,TRO,<NA>,SS,1,1,24,1,3,2,0,<NA>,<NA>,<NA>,<NA>,<NA>,Abercrombie,Frank,110018,aberd101,1000017,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
1,addybo01,1871,1,RC1,<NA>,2B,22,22,606,67,72,42,5,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
2,addybo01,1871,1,RC1,<NA>,SS,3,3,96,8,14,7,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
3,addybo01,1873,1,PH2,<NA>,2B,10,<NA>,249,24,23,8,2,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
4,addybo01,1873,2,BS1,<NA>,OF,31,<NA>,843,30,3,14,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256727,zimmebr01,2021,1,CLE,AL,<NA>,99,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,15,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,299,44,68,9,1,8,35,30,122,0,15,0,4,3,right_only
256728,zimmebr02,2021,1,BAL,AL,<NA>,14,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,0,0,0,0,0,0,0,3,0,0,0,0,0,right_only
256729,zimmeky01,2021,1,KCA,AL,<NA>,52,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,0,0,0,0,right_only
256730,zimmery01,2021,1,WAS,NL,<NA>,110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,255,27,62,16,0,14,46,16,77,0,0,0,2,9,right_only


## Init Pitching data

In [221]:
# pitching stats by year 
pitching = pd.DataFrame(pitching()) # this is fine

sleep(1)

pitching = pitching.convert_dtypes()

In [222]:
pitching = pitching.rename(columns={"playerID": "key_bbref"})

In [223]:
a = np.intersect1d(df.columns, pitching.columns)
print(list(a))

['G', 'GS', 'WP', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID']


In [224]:
df = df.merge(pitching, on=[ 'G', 'GS', 'WP', 'key_bbref', 'lgID', 'stint', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator="second_merge")



## FIP, wRC+

In [225]:
# bwar_bat stats by year 
bwar_bat = pd.DataFrame(bwar_bat()) # this is fine

bwar_bat = bwar_bat.convert_dtypes()


In [226]:
bwar_bat = bwar_bat.rename(columns={
    "year_ID": "yearID",
    "player_ID": "key_bbref",
    "team_ID": "teamID",
    "lg_ID": "lgID",
})

In [227]:
a = np.intersect1d(df.columns, bwar_bat.columns)
print(list(a))

['G', 'key_bbref', 'lgID', 'teamID', 'yearID']


In [228]:
df = df.merge(bwar_bat, on=['G', 'key_bbref', 'lgID', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator="third_merge")

In [229]:
# bwar_pit stats by year 
bwar_pitch = pd.DataFrame(bwar_pitch()) # this is fine

bwar_pitch = bwar_pitch.convert_dtypes()


In [230]:
bwar_pitch = bwar_pitch.rename(columns={
    "year_ID": "yearID",
    "player_ID": "key_bbref",
    "team_ID": "teamID",
    "lg_ID": "lgID",
})

In [231]:
a = np.intersect1d(df.columns, bwar_pitch.columns)
print(list(a))

['G', 'GS', 'WAA', 'WAR', 'WAR_rep', 'key_bbref', 'lgID', 'mlb_ID', 'name_common', 'salary', 'stint_ID', 'teamID', 'yearID']


In [232]:
df = df.merge(bwar_pitch, on=['G', 'GS', 'WAA', 'WAR', 'WAR_rep', 'key_bbref', 'lgID', 'mlb_ID', 'name_common', 'salary', 'stint_ID', 'teamID', 'yearID'], how='outer', validate='many_to_many', indicator="fourth_merge")
df

,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,BMI,bats,throws,W,L,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,HBP,BK,BFP,GF,R,SH,SF,GIDP,second_merge,name_common,mlb_ID,stint_ID,pitcher,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,third_merge,RA,xRA,BIP,BIP_perc,ERA_plus,WAA_adj,fourth_merge
0,abercda01,1871,1,TRO,<NA>,SS,1,1,24,1,3,2,0,<NA>,<NA>,<NA>,<NA>,<NA>,Abercrombie,Frank,110018,aberd101,1000017,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,Frank Abercrombie,110018,1,N,4,<NA>,-1.3,-1.3,0.1,0.01,-0.08,-0.07,both,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
1,addybo01,1871,1,RC1,<NA>,2B,22,22,606,67,72,42,5,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
2,addybo01,1871,1,RC1,<NA>,SS,3,3,96,8,14,7,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
3,addybo01,1873,1,PH2,<NA>,2B,10,<NA>,249,24,23,8,2,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
4,addybo01,1873,2,BS1,<NA>,OF,31,<NA>,843,30,3,14,0,<NA>,<NA>,<NA>,<NA>,<NA>,Addy,Bob,110074,addyb101,1000070,1842,1910,160,68,24.327647,L,L,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351906,strasst01,2022,<NA>,WSN,NL,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Stephen Strasburg,544931,1,<NA>,<NA>,35000000,<NA>,<NA>,<NA>,0.044,-0.3143,-0.28,NaN,7,2.205,15,0.0064,31.571429,-0.0051,right_only
351907,tetreja01,2022,<NA>,WSN,NL,<NA>,4,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Jackson Tetreault,676194,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.1967,-0.3812,-0.21,NaN,15,10.559,77,0.0327,77.933333,-0.023,right_only
351908,thompma02,2022,<NA>,WSN,NL,<NA>,6,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Mason Thompson,666168,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0485,0.2664,0.2,NaN,0,2.712,12,0.0051,<NA>,-0.1145,right_only
351909,vothau01,2022,<NA>,WSN,NL,<NA>,19,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>

### Clean-up

In [233]:
all_players = df.drop_duplicates(subset=['key_bbref'])

In [235]:
everyone = df[
    [
        "key_bbref",
        "name_last",
        "name_first",
        "weight",
        "height",
        "BMI",
        "yearID",
        "POS",
        "WAR",
        "WAA",
        "pitcher",
        "ERA_plus",
        "birthYear",
        "deathYear",
        "teamID",
        "lgID",
        "runs_above_avg",
        "runs_above_avg_def",
        "WAA_adj",
        "G",
        "GS",
        "InnOuts",
        "PO",
        "A",
        "E",
        "DP",
        "PB",
        "WP",
        "SB",
        "CS",
        "ZR",
        "bats",
        "throws",
        "R",
        "H",
        "HR",
        "BB",
        "SO",
        "IBB",
        "HBP",
        "SH",
        "SF",
        "GIDP",
        "W",
        "L",
        "CG",
        "SHO",
        "SV",
        "IPouts",
        "ER",
        "BAOpp",
        "ERA",
        "BK",
        "BFP",
        "GF",
        "name_common",
        "mlb_ID",
        "stint_ID",
        "PA",
        "salary",
        "runs_above_avg_off",
        "WAR_rep",
        "RA",
        "xRA",
        "BIP",
        "BIP_perc",
        "second_merge",
        "third_merge",
        "fourth_merge",
        "key_mlbam",
        "key_retro",
        "key_fangraphs",
    ]
]

In [237]:

df = everyone[
    [
        "key_bbref",
        "weight",
        "height",
        "BMI",
        "name_last",
        "name_first",
        "POS",
        "yearID",
        "birthYear",
        "deathYear",
        "teamID",
        "lgID",
        "R",
        "H",
        "HR",
        "BB",
        "SO",
        "WAR",
        "WAA",
        "pitcher",
        "ERA_plus",
    ]
]

# Here it is:

In [238]:
df = df.sort_values('weight', ascending=False)

In [239]:
df

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,R,H,HR,BB,SO,WAR,WAA,pitcher,ERA_plus
164878,youngwa01,320,77,37.946042,Young,Walter,1B,2005,1980,2015,BAL,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
184576,diazju03,315,76,38.342579,Diaz,Jumbo,P,2015,1984,<NA>,CIN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
184578,diazju03,315,76,38.342579,Diaz,Jumbo,P,2016,1984,<NA>,CIN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,-0.01,-0.01,Y,<NA>
184580,diazju03,315,76,38.342579,Diaz,Jumbo,P,2017,1984,<NA>,TBA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
184574,diazju03,315,76,38.342579,Diaz,Jumbo,P,2014,1984,<NA>,CIN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351906,strasst01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2022,<NA>,<NA>,WSN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,-0.28,-0.3143,<NA>,31.571429
351907,tetreja01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2022,<NA>,<NA>,WSN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,-0.21,-0.3812,<NA>,77.933333
351908,thompma02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2022,<NA>,<NA>,WSN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,0.2,0.2664,<NA>,<NA>
351909,vothau01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2022,<NA>,<NA>,WSN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,-0.81,-1.2578,<NA>,39.67619


In [240]:
df = df.drop_duplicates(subset=['key_bbref'])

In [241]:
df = df.convert_dtypes()

In [242]:
df

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,R,H,HR,BB,SO,WAR,WAA,pitcher,ERA_plus
164878,youngwa01,320,77,37.946042,Young,Walter,1B,2005,1980,2015,BAL,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
184576,diazju03,315,76,38.342579,Diaz,Jumbo,P,2015,1984,<NA>,CIN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
44051,brownju01,295,76,35.90813,Brown,Jumbo,P,1933,1907,1966,NYA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
144186,youngdm01,295,74,37.875339,Young,Dmitri,1B,1997,1973,<NA>,SLN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
157446,rauchjo01,290,83,29.59646,Rauch,Jon,P,2002,1978,<NA>,CHA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298214,zomphch01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1926,<NA>,<NA>,CEL,NNL,<NA>,<NA>,<NA>,<NA>,<NA>,-0.05,-0.35,N,<NA>
351180,bellobr01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2022,<NA>,<NA>,BOS,AL,<NA>,<NA>,<NA>,<NA>,<NA>,-0.08,-0.1094,<NA>,50.875
351261,hillga02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2022,<NA>,<NA>,DET,AL,<NA>,<NA>,<NA>,<NA>,<NA>,-0.08,-0.1696,<NA>,69.642857
351467,ogandcr01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2022,<NA>,<NA>,TBR,AL,<NA>,<NA>,<NA>,<NA>,<NA>,0.01,-0.0033,<NA>,95.7


In [243]:
df.describe().to_csv('mlb_all_stats.csv')

After some noodling, this is the best combo of BMI:height that I could find such that we have enough players at each position to fill out a roster. 2B and 1B ended up being a bottleneck, which isn't surprising. Second-basemen are typically the smallest player on the team, and First-basemen are almost universally the tallest fielder on the roster.

In [244]:
tryouts = df[ # Let's separate the meat from the chaff
    (df.BMI >= df.BMI.quantile(0.99))
    & (df.height <= df.height.quantile(0.23))]

In [246]:
tryouts.to_csv('tryouts.csv')

In [965]:
tryouts.POS.value_counts()

P     17
C     15
1B     8
OF     7
2B     3
SS     1
3B     1
Name: POS, dtype: Int64

In [966]:
# Probably easiest if we make ourselves a dataframe for every position, since we'll need to fill up a 26-man roster, which is parsed into several discrete roles that each require a particular number of players. 

# These role counts aren't codified, but teams have nearly always carried the same distribution of player-roles.

# The standard breakdown is:
# 13 Pitchers (5x SP, 7x RP)
# 5 OF
# 2 C
# 4 IF
# 2 Utility

# For Utility players, we'll likely take two extra middle infielders, because middle infielders can usually play any field position without looking like they have no idea what they're doing. E.g., you can put a shortstop at first base, and he'll be a pretty mediocre-to-bad first baseman — but if you put a first baseman at shortstop, he's likely to spend the entire game crying.

tryouts_P = tryouts[tryouts['POS'] == "P"]
tryouts_C = tryouts[tryouts['POS'] == "C"]
tryouts_1B = tryouts[tryouts['POS'] == "1B"]
tryouts_2B = tryouts[tryouts['POS'] == "2B"]
tryouts_3B = tryouts[tryouts['POS'] == "3B"]
tryouts_SS = tryouts[tryouts['POS'] == "SS"]
tryouts_OF = tryouts[tryouts['POS'] == "OF"]

In [967]:
huskies_P = (
    tryouts_P.sort_values("BMI", ascending=False)
    .nlargest(13, "BMI")
)  # 13 heaviest P
huskies_P

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
182432,colonba01,285,71,39.748992,Colon,Bartolo,P,2011,1973,<NA>,NYA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
243208,moronre01,265,70,38.02312,Moronta,Reyes,P,2017,1993,<NA>,SFN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
216625,mijarjo01,265,71,36.959589,Mijares,Jose,P,2011,1984,<NA>,MIN,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3994,healeto01,155,55,36.025023,Healey,Tom,P,1878,1853,1891,PRO,NL,9,0,2,1,0,0,2,0,4,-0.02,-0.02,Y,<NA>
229336,machije01,257,71,35.843828,Machi,Jean,P,2014,1982,<NA>,SFN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
199068,rodnefe01,240,71,33.472835,Rodney,Fernando,P,2002,1977,<NA>,DET,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
236741,castran01,240,71,33.472835,Castro,Angel,P,2015,1982,<NA>,OAK,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
248116,colined01,240,71,33.472835,Colina,Edwar,P,2020,1997,<NA>,MIN,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
245959,valdefr01,239,71,33.333365,Valdez,Framber,P,2021,1993,<NA>,HOU,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
224531,carigan01,235,71,32.775484,Carignan,Andrew,P,2012,1986,<NA>,OAK,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [968]:
huskies_P = (
    tryouts_P.sort_values("BMI", ascending=False)
    .nlargest(13, "BMI")
)  # 13 heaviest P
huskies_P

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
182432,colonba01,285,71,39.748992,Colon,Bartolo,P,2011,1973,<NA>,NYA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
243208,moronre01,265,70,38.02312,Moronta,Reyes,P,2017,1993,<NA>,SFN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
216625,mijarjo01,265,71,36.959589,Mijares,Jose,P,2011,1984,<NA>,MIN,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3994,healeto01,155,55,36.025023,Healey,Tom,P,1878,1853,1891,PRO,NL,9,0,2,1,0,0,2,0,4,-0.02,-0.02,Y,<NA>
229336,machije01,257,71,35.843828,Machi,Jean,P,2014,1982,<NA>,SFN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
199068,rodnefe01,240,71,33.472835,Rodney,Fernando,P,2002,1977,<NA>,DET,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
236741,castran01,240,71,33.472835,Castro,Angel,P,2015,1982,<NA>,OAK,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
248116,colined01,240,71,33.472835,Colina,Edwar,P,2020,1997,<NA>,MIN,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
245959,valdefr01,239,71,33.333365,Valdez,Framber,P,2021,1993,<NA>,HOU,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
224531,carigan01,235,71,32.775484,Carignan,Andrew,P,2012,1986,<NA>,OAK,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [969]:
huskies_C = (
    tryouts_C.sort_values("BMI", ascending=False)
    .nlargest(2, "BMI")
)  # 2 heaviest C
huskies_C

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
248415,kirkal01,245,68,37.25171,Kirk,Alejandro,C,2021,1998,<NA>,TOR,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
207632,penabr01,240,69,35.441412,Pena,Brayan,C,2016,1982,<NA>,SLN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [970]:
huskies_1B = (
    tryouts_1B.sort_values("BMI", ascending=False)
    .nlargest(1, "BMI")
)  # 1 heaviest 1B
huskies_1B

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
217568,sandopa01,268,70,38.45357,Sandoval,Pablo,1B,2010,1986,<NA>,SFN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [971]:
huskies_2B = (
    tryouts_2B.sort_values("BMI", ascending=False)
    .nlargest(3, "BMI")
)  # 3 heaviest 2B (1x starter, 2x utility)
huskies_2B

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
241635,whitety01,238,71,33.193895,White,Tyler,2B,2017,1990,<NA>,HOU,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
232566,phegljo01,225,70,32.283781,Phegley,Josh,2B,2013,1988,<NA>,CHA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
230370,solando01,210,68,31.930037,Solano,Donovan,2B,2014,1987,<NA>,MIA,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [972]:
huskies_SS = (
    tryouts_SS.sort_values("BMI", ascending=False)
    .nlargest(1, "BMI")
)  # 1 heaviest SS
huskies_SS

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
184377,tejadmi01,220,69,32.487961,Tejada,Miguel,SS,1997,1974,<NA>,OAK,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-0.23,-0.59,N,<NA>


In [973]:
huskies_3B = (
    tryouts_3B.sort_values("BMI", ascending=False)
    .nlargest(1, "BMI")
)  # 1 heaviest 3B
huskies_3B

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
208969,callaal01,210,68,31.930037,Callaspo,Alberto,3B,2010,1983,<NA>,KCA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [974]:
huskies_OF = (
    tryouts_OF.sort_values("BMI", ascending=False)
    .nlargest(5, "BMI")
)  # 5 heaviest OF
huskies_OF

,key_bbref,weight,height,BMI,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus
247295,naylojo01,250,71,34.867537,Naylor,Josh,OF,2020,1997,<NA>,SDN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
223992,vicieda01,240,71,33.472835,Viciedo,Dayán,OF,2012,1989,<NA>,CHA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
247419,ramirha02,232,70,33.288165,Ramirez,Harold,OF,2021,1994,<NA>,CLE,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
244258,astudwi01,225,69,33.226324,Astudillo,Willians,OF,2021,1991,<NA>,MIN,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
51268,fothebo01,230,70,33.001198,Fothergill,Bob,OF,1923,1897,1938,DET,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [975]:
huskies = pd.concat([
    huskies_P,
    huskies_C,
    huskies_1B,
    huskies_2B,
    huskies_3B,
    huskies_SS,
    huskies_OF
])

In [976]:
huskies.describe()

,weight,height,BMI,yearID,birthYear,deathYear,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,ERA_plus
count,26.000000,26.000000,26.000000,26.000000,26.000000,2.000000,1,1,1,1,1,1,1,1,1,2.000000,2.000000,0
mean,236.500000,69.576923,34.299658,2005.153846,1978.076923,1914.500000,9.0,0.0,2.0,1.0,0.0,0.0,2.0,0.0,4.0,-0.125000,-0.305000,<NA>
std,24.081943,3.151800,2.208648,32.024606,31.757737,33.234019,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.148492,0.403051,<NA>
min,155.000000,55.000000,31.930037,1878.000000,1853.000000,1891.000000,9,0,2,1,0,0,2,0,4,-0.230000,-0.590000,<NA>
25%,226.250000,69.250000,32.775484,2010.000000,1982.000000,1902.750000,9,0,2,1,0,0,2,0,4,-0.177500,-0.447500,<NA>
50%,238.500000,70.500000,33.403100,2013.500000,1986.000000,1914.500000,9,0,2,1,0,0,2,0,4,-0.125000,-0.305000,<NA>
75%,243.750000,71.000000,35.743224,2019.250000,1992.500000,1926.250000,9,0,2,1,0,0,2,0,4,-0.072500,-0.162500,<NA>
max,285.000000,71.000000,39.748992,2021.000000,1998.000000,1938.000000,9,0,2,1,0,0,2,0,4,-0.020000,-0.020000,<NA>


In [ ]:
huskies

,key_bbref,weight,height,name_last,name_first,POS,yearID,birthYear,deathYear,teamID,lgID,AB,R,H,2B,3B,HR,RBI,BB,SO,WAR,WAA,pitcher,ERA_plus,KG,meters,BMI,ratio
182432,colonba01,285,71,Colon,Bartolo,P,2011,1973,<NA>,NYA,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,129.27372,1.8034,39.748992,71.683331
243208,moronre01,265,70,Moronta,Reyes,P,2017,1993,<NA>,SFN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,120.20188,1.778,38.02312,67.605107
216625,mijarjo01,265,71,Mijares,Jose,P,2011,1984,<NA>,MIN,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,120.20188,1.8034,36.959589,66.652922
3994,healeto01,155,55,Healey,Tom,P,1878,1853,1891,PRO,NL,9,0,2,1,0,0,2,0,4,-0.02,-0.02,Y,<NA>,70.30676,1.397,36.025023,50.326958
229336,machije01,257,71,Machi,Jean,P,2014,1982,<NA>,SFN,NL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,116.573144,1.8034,35.843828,64.640759
199068,rodnefe01,240,71,Rodney,Fernando,P,2002,1977,<NA>,DET,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,108.86208,1.8034,33.472835,60.364911
236741,castran01,240,71,Castro,Angel,P,2015,1982,<NA>,OAK,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,108.86208,1.8034,33.472835,60.364911
248116,colined01,240,71,Colina,Edwar,P,2020,1997,<NA>,MIN,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,108.86208,1.8034,33.472835,60.364911
245959,valdefr01,239,71,Valdez,Framber,P,2021,1993,<NA>,HOU,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,108.408488,1.8034,33.333365,60.11339
224531,carigan01,235,71,Carignan,Andrew,P,2012,1986,<NA>,OAK,AL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,106.59412,1.8034,32.775484,59.107308


# Let's run this through Baseball Reference offline
They've got handy tool to produce an expected season for a given player. These are the CSVs that it spits out.

In [ ]:
batters = pd.read_csv("../data/huskies_batters.csv")
pitchers = pd.read_csv("../data/huskies_pitchers.csv")

Time to take a look at our team

In [ ]:
# Average Husky WAR
(pitchers.WAR.sum() + batters.WAR.sum()) / 26

1.2153846153846155

In [ ]:
# Total Husky WAR
husky_war_sum = (pitchers.WAR.sum() + batters.WAR.sum())
# Average Husky height
husky_height_mean = (pitchers.height.sum() + batters.height.sum()) / 26
# Average Husky weight
husky_weight_mean = (pitchers.weight.sum() + batters.weight.sum()) / 26

In [ ]:
# Average weight of all players
everyone_weight_mean = everyone.weight.mean()
# Average height of all players
everyone_height_mean = everyone.height.mean()

In [ ]:
pitchers.WAR.sum()

13.899999999999999

In [ ]:
batters.describe(
)

,height,weight,G,PA,AB,R,H,double,triple,homer,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Rbat,Rbaser,Rdp,Rfield,Rpos,RAA,WAA,Rrep,RAR,WAR,waaWL%,162WL%,oWAR,dWAR,oRAR,outs,walks,1B,2B,3B,HR,OUT,WALK,sum
count,13.000000,13.000000,13.0,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,12.0,13.000000,13.000000,12.000000,12.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,69.538462,233.307692,162.0,556.307692,509.076923,60.461538,137.307692,27.384615,1.923077,14.153846,66.769231,2.461538,1.923077,35.692308,79.923077,0.269308,0.320692,0.413231,0.733923,98.846154,211.000000,15.0,5.153846,1.923077,4.583333,2.083333,0.615385,-1.461538,-0.615385,-3.615385,0.000000,-5.230769,-0.507692,19.384615,14.384615,1.361538,0.496692,0.498923,1.723077,-0.315385,18.000000,329.000000,37.615385,0.262531,0.049272,0.003466,0.025108,0.592548,0.067074,0.737469
std,1.126601,16.188394,0.0,53.658464,49.358318,13.684635,20.093053,4.407162,1.977437,6.375031,14.359398,1.941451,2.059998,13.780328,34.731018,0.025227,0.028946,0.039368,0.062551,16.118551,33.662541,4.0,2.703274,3.593085,0.792961,1.831955,10.882237,1.898042,1.192928,3.014920,6.244998,13.766831,1.412717,2.063107,14.505967,1.476222,0.008873,0.005155,1.314758,0.838497,12.529964,34.285565,14.534662,0.028998,0.006912,0.003764,0.010373,0.045736,0.024663,0.028998
min,68.000000,210.000000,162.0,495.000000,459.000000,35.000000,105.000000,19.000000,0.000000,5.000000,42.000000,0.000000,0.000000,10.000000,24.000000,0.225000,0.268000,0.351000,0.649000,76.000000,161.000000,9.0,1.000000,0.000000,3.000000,0.000000,-14.000000,-6.000000,-2.000000,-8.000000,-9.000000,-19.000000,-1.800000,17.000000,-2.000000,-0.200000,0.488000,0.491000,0.300000,-1.500000,5.000000,272.000000,10.000000,0.208333,0.037773,0.000000,0.009524,0.495446,0.019881,0.688525
25%,69.000000,225.000000,162.0,521.000000,478.000000,55.000000,126.000000,24.000000,1.000000,8.000000,62.000000,1.000000,1.000000,27.000000,54.000000,0.257000,0.299000,0.381000,0.697000,91.000000,192.000000,12.5,3.000000,0.000000,4.000000,1.000000,-4.000000,-3.000000,-2.000000,-5.000000,-6.000000,-15.000000,-1.600000,18.000000,5.000000,0.400000,0.490000,0.496000,0.900000,-1.100000,10.000000,309.000000,29.000000,0.240809,0.045537,0.001988,0.013652,0.568015,0.054726,0.720137
50%,70.000000,232.000000,162.0,544.000000,481.000000,59.000000,139.000000,28.000000,2.000000,15.000000,68.000000,2.000000,1.000000,32.000000,89.000000,0.270000,0.320000,0.410000,0.725000,97.000000,206.000000,14.5,5.000000,0.000000,5.000000,2.000000,-1.000000,-1.000000,-1.000000,-3.000000,1.000000,-10.000000,-0.800000,19.000000,8.000000,0.900000,0.494000,0.497000,1.500000,-0.200000,16.000000,331.000000,33.000000,0.261383,0.050946,0.003317,0.029412,0.587332,0.061420,0.738617
75%,70.000000,240.000000,162.0,586.000000,541.000000,66.000000,150.000000,30.000000,2.000000,18.000000,71.000000,3.000000,2.000000,42.000000,98.000000,0.280000,0.330000,0.443000,0.773000,108.000000,229.000000,17.5,7.000000,2.000000,5.000000,2.250000,8.000000,0.000000,0.000000,-2.000000,5.000000,0.000000,0.000000,20.000000,20.000000,2.100000,0.500000,0.502000,2.300000,0.400000,23.000000,339.000000,48.000000,0.279863,0.052277,0.003413,0.031716,0.624242,0.080944,0.759191
max,71.000000,268.000000,162.0,687.000000,629.000000,92.000000,180.000000,35.000000,8.000000,23.000000,97.000000,6.000000,8.000000,58.000000,132.000000,0.325000,0.374000,0.480000,0.854000,136.000000,287.000000,21.0,9.000000,13.000000,6.000000,6.000000,27.000000,1.000000,1.000000,3.000000,9.000000,29.000000,3.100000,24.000000,48.000000,4.900000,0.519000,0.508000,4.600000,1.000000,44.000000,394.000000,59.000000,0.311475,0.062857,0.015238,

In [ ]:
batters['total_average'] = ((batters['TB'] + batters['HBP'] + batters['BB'] + batters['SB'] - batters['CS']) / (batters['AB'] - batters['H'] + batters['CS'] + batters['GDP']))

In [ ]:
batters.WAR.sum()

17.700000000000003

In [ ]:
17.7 + 13.9

31.6

In [ ]:
pitchers.median()

/var/folders/m9/kw6252cj3lqgftmsg97nw4z80000gn/T/ipykernel_57407/528101122.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pitchers.median()


height      71.0000
weight     240.0000
W            4.0000
L            5.0000
W-L%         0.4665
Dec          3.6800
ERA         68.0000
G            0.0000
GS          13.0000
CG           0.0000
SHO          0.0000
SV           0.0000
IP          67.0000
H          109.0000
R           42.0000
ER          40.0000
HR          14.0000
BB          48.0000
IBB          2.0000
SO          67.0000
HBP          3.0000
BK           0.0000
WP           4.0000
BF         413.0000
ERA+▼      109.0000
FIP          4.1500
WHIP         1.3560
H9           8.9000
HR9          0.8000
BB9          3.8000
SO9          7.3000
SO/BB        2.2000
IP.1        67.0000
G.1         68.0000
GS.1         0.0000
R.1         42.0000
RA9          4.1800
RA9opp       4.6800
RA9def       0.0600
RA9role     -0.3100
PPFp        95.8000
RA9avg       4.6700
RAA          4.0000
WAA          0.1000
gmLI         1.0000
WAAadj       0.0000
WAR          0.5000
RAR          6.0000
waaWL%       0.5060
162WL%       0.5000


In [ ]:
pitchers.WAR.sum()

13.899999999999999

In [ ]:
people.shape

(19729, 24)

In [ ]:
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981,12,27,USA,CO,Denver,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,David,Aardsma,David Allan,215,75,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,2021,1,22,USA,GA,Atlanta,Hank,Aaron,Henry Louis,180,72,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954,9,8,USA,CA,Orange,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Don,Aase,Donald William,190,75,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972,8,25,USA,FL,Palm Beach,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Andy,Abad,Fausto Andres,184,73,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [ ]:
people.weight.describe()

count    19729.000000
mean       188.133712
std         22.495432
min         65.000000
25%        172.000000
50%        185.000000
75%        200.000000
max        320.000000
Name: weight, dtype: float64

In [ ]:
comparison = pd.DataFrame()

In [ ]:
comparison['everyone_weight'] = people.weight.describe()
comparison['huskies_weight'] = huskies.weight.describe()
comparison['delta_weight'] = comparison['huskies_weight'] - comparison['everyone_weight']
comparison['everyone_height'] = people.height.describe()
comparison['huskies_height'] = huskies.height.describe()
comparison['delta_height'] = comparison['huskies_height'] - comparison['everyone_height']


In [ ]:
comparison = comparison.convert_dtypes()
comparison = comparison.drop(labels='count')

In [ ]:
statcompare = pd.DataFrame()
statcompare['huskies_weight'] = 

In [ ]:
comparison.to_csv('body_comparison.csv')

In [ ]:
stats = pd.DataFrame()

In [ ]:
batters.columns

Index(['player', 'height', 'weight', 'G', 'PA', 'AB', 'R', 'H', 'double',
       'triple', 'homer', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG',
       'OPS', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'Pos', 'Rbat',
       'Rbaser', 'Rdp', 'Rfield', 'Rpos', 'RAA', 'WAA', 'Rrep', 'RAR', 'WAR',
       'waaWL%', '162WL%', 'oWAR', 'dWAR', 'oRAR', 'outs', 'walks', '1B', '2B',
       '3B', 'HR', 'OUT', 'WALK', 'sum', 'total_average'],
      dtype='object')

In [900]:
everyone.convert_dtypes()
everyone_bio = everyone[['height', 'weight', 'BMI', 'WAR']]
everyone.describe().to_csv('everyone_describe.csv')
huskies.convert_dtypes()
huskies.describe().to_csv('huskies_describe.csv')


KeyError: "['BMI'] not in index"

In [701]:
brefids = huskies.key_bbref.to_list()

In [702]:
brefids

['colonba01',
 'moronre01',
 'mijarjo01',
 'healeto01',
 'machije01',
 'rodnefe01',
 'castran01',
 'colined01',
 'valdefr01',
 'carigan01',
 'akinke01',
 'mateoju01',
 'gonzaen01',
 'kirkal01',
 'penabr01',
 'sandopa01',
 'whitety01',
 'phegljo01',
 'solando01',
 'callaal01',
 'tejadmi01',
 'naylojo01',
 'vicieda01',
 'ramirha02',
 'astudwi01',
 'fothebo01']

In [801]:
all_players.columns

Index(['key_bbref', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS',
       'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR',
       'name_last', 'name_first', 'key_mlbam', 'key_retro', 'key_fangraphs',
       'birthYear', 'deathYear', 'weight', 'height', 'bats', 'throws', 'AB',
       'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF',
       'GIDP', '_merge', 'W', 'L', 'CG', 'SHO', 'SV', 'IPouts', 'ER', 'BAOpp',
       'ERA', 'BK', 'BFP', 'GF', 'second_merge', 'name_common', 'mlb_ID',
       'stint_ID', 'pitcher', 'PA', 'salary', 'runs_above_avg',
       'runs_above_avg_off', 'runs_above_avg_def', 'WAR_rep', 'WAA', 'WAR',
       'third_merge', 'RA', 'xRA', 'BIP', 'BIP_perc', 'ERA_plus', 'WAA_adj',
       'fourth_merge'],
      dtype='object')

In [ ]:
all_players['KG'] = all_players['weight'] * KG_TO_LB
all_players['meters'] = all_players['height'] * height_

In [819]:
all_players['BMI'] = all_players['KG'] / all_players['meters'] ** 2

KeyError: 'KG'

In [821]:
all_players_describe = all_players.describe()
huskies_describe = huskies.describe()

In [822]:
all_players_describe = all_players_describe.join(huskies_describe, how='left', lsuffix='_mlb', rsuffix='_huskies')

In [824]:
all_players_describe = all_players_describe.convert_dtypes()
huskies_describe = huskies_describe.convert_dtypes()

In [825]:
all_players_describe.to_csv('stat_comparison.csv')
huskies_describe.to_csv('huskie_bodies.csv')
